In [29]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from typing import Union
import numpy as np
import pandas as pd

In [30]:
ModelRegressor = Union[RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, LogisticRegression]

In [35]:
def dummy_fit_predict(classifier: ModelRegressor, X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.DataFrame):
    if X_train.empty or X_test.empty or y_train.empty:
        raise ValueError("Empty set passed!")
    
    classifier.fit(X_train, y_train)
    pred_train = classifier.predict_proba(X_train)
    pred_test = classifier.predict_proba(X_test).fillna(0)


<h1> 1. Wczytanie przetworzonych zbiorow danych z Lab3. </h1>

In [20]:
X_train = pd.read_csv('X_train.csv', usecols = [i for i in range(1,7)])
X_test = pd.read_csv('X_test.csv', usecols = [i for i in range(1,7)])
y_train = pd.read_csv('y_train.csv', usecols = [1])
y_test = pd.read_csv('y_test.csv', usecols = [1])

In [22]:
X_train.head(10)

,sex,cabin,CabinReduced,sex_mapped,cabin_mapped,CabinReduced_mapped
0,female,NaN,NaN,0,0,0
1,female,NaN,NaN,0,0,0
2,female,NaN,NaN,0,0,0
3,male,NaN,NaN,1,0,0
4,female,NaN,NaN,0,0,0
5,male,NaN,NaN,1,0,0
6,female,E36,E,0,1,1
7,female,NaN,NaN,0,0,0
8,male,C68,C,1,2,2
9,male,E24,E,1,3,1


<h1> 2. Charakterystyka algorytmu Random Forests. </h1>

<h1> 3. Uzycie algorytmu Random Forests </h1>

NameError: name 'Union' is not defined